In [1]:
import numpy as np

First, let's derive the inverse X matrix that maps from fluxes to "a" coefficients:

In [2]:
X = np.vander([-1, 0, 1], 3)
np.linalg.inv(X)

array([[ 0.5, -1. ,  0.5],
       [-0.5,  0. ,  0.5],
       [ 0. ,  1. ,  0. ]])

Then, let's propagate the uncertainties:

In [4]:
np.linalg.inv(np.dot(X.T, X))

array([[ 1.5,  0. , -1. ],
       [ 0. ,  0.5,  0. ],
       [-1. ,  0. ,  1. ]])

This means that a_1 and a_2 are independent with variances 0.5 and 1.5 respectively.